## 分会议建立bulk_json

In [25]:
from utils import read_pdf, listdir, org_years_dict
import json, os
from tqdm import tqdm_notebook 
from multiprocessing import Pool
import time, traceback

In [24]:
file_paths = []
listdir('/home/d/papers_data/', file_paths)
len(file_paths)

18810

# bulk insert!(ok)

In [33]:
! curl -X GET "localhost:9200/_cat/indices?v"

health status index uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   iclr  wdQPpCI5TFq1uGwUslVj_w   1   1       1980            8    178.2mb        178.2mb
yellow open   bank  U8Pq9fF_SO-qmNwuVGYzLg   1   1       1000            0    414.3kb        414.3kb
yellow open   aaai  cjDr4muCRA-_MXyxSbSjag   1   1       5062            8    213.9mb        213.9mb
yellow open   emnlp hqBbqnSuT82-ZFPhixb-4A   1   1       3245            1    101.9mb        101.9mb
yellow open   icml  RihCp80hTEu_K9ij7UEPkw   1   1       1054            0       42mb           42mb
yellow open   kdd   iRGYz-AMR_-YmL15Oa4OAA   1   1        828            5     41.8mb         41.8mb
yellow open   acl   gCWVLshARBm0lsQe8fUVNA   1   1       3434            2    107.9mb        107.9mb
yellow open   nips  ClZZkJ0RQZ21EARwV8fO_A   1   1       3114            7      114mb          114mb


In [35]:
1980+5062+3245+1054+ 828+3434+3114
# 应该要是18724-3

18717

In [30]:
2+0+2+0+5+1+8+1

19

In [8]:
from utils import org_years_dict
json_file_folder = './pdf_es_json'
json_file_paths = []
for org in org_years_dict.keys():
    file_name = json_file_folder + f'/{org}_' + '_'.join([year for year in org_years_dict[org]]) + '.json'
    json_file_paths.append(file_name)

json_file_paths

['./pdf_es_json/ACL_2014_2015_2016_2017_2018_2019.json',
 './pdf_es_json/AAAI_2014_2015_2016_2017_2018_2019.json',
 './pdf_es_json/EMNLP_2014_2015_2016_2017_2018_2019.json',
 './pdf_es_json/ICLR_2017_2018_2019.json',
 './pdf_es_json/ICML_2017_2018.json',
 './pdf_es_json/KDD_2017_2018_2019.json',
 './pdf_es_json/NIPS_2017_2018_2019.json']

In [9]:
import json, requests
from utils import org_years_dict

def bulk_insert2ES(org, bulk_size = 300, json_file_folder = './pdf_es_json'):
    json_file_paths = []
    for _org in org_years_dict.keys():
        file_name = json_file_folder + f'/{_org}_' + '_'.join([year for year in org_years_dict[_org]]) + '.json'
        json_file_paths.append(file_name)
    
    for path in json_file_paths:
        if path.split('/')[-1].split('_')[0] != org: continue
    
        with open(path, 'r') as f1:
            bulk_content_lines = f1.readlines()
        
        url = f'http://localhost:9200/{org.lower()}/_bulk?pretty&refresh'
        headers = {'Content-Type': 'application/json'}
        for _index in range(0, len(bulk_content_lines), bulk_size*2):
            data = ''.join(bulk_content_lines[_index: _index + bulk_size*2]) + '\n'
#             print(len(data))
            r = requests.post(url = url, headers = headers, data=data)
#             j = json.loads(r.text)
#             print(j)
#             return j
#             break
    print(f'{org} DONE!')
    return True

In [10]:
bulk_insert2ES(org = 'ACL', bulk_size = 300)

ACL DONE!


True

In [11]:
bulk_insert2ES(org = 'EMNLP', bulk_size = 300)

EMNLP DONE!


True

In [12]:
bulk_insert2ES(org = 'AAAI', bulk_size = 300)

AAAI DONE!


True

In [13]:
bulk_insert2ES(org = 'ICLR', bulk_size = 300)

ICLR DONE!


True

In [14]:
bulk_insert2ES(org = 'ICML', bulk_size = 300)

ICML DONE!


True

In [15]:
bulk_insert2ES(org = 'KDD', bulk_size = 300)

KDD DONE!


True

In [16]:
bulk_insert2ES(org = 'NIPS', bulk_size = 300)

NIPS DONE!


True

## 查询(失败)

In [58]:
def _base_script(key_word:str):
    base_script = {}
    base_script["key_words_frequency"] = {}
    base_script["key_words_frequency"]["script"] = {}
    base_script["key_words_frequency"]["script"]["lang"] = "painless"
    base_script["key_words_frequency"]["script"]["source"] = f"""
    int count = 0;
    if(doc['Text.keyword'].size() > 0 && doc['Text.keyword'].value.indexOf(params.phrase)!=-1) count++;
    return count;        
    """
    base_script["key_words_frequency"]["script"]["params"] = {}
    base_script["key_words_frequency"]["script"]["params"]["phrase"] = key_word

    return base_script

In [59]:
x = _base_script('represent')
x

{'key_words_frequency': {'script': {'lang': 'painless',
   'source': "\n    int count = 0;\n    if(doc['Text.keyword'].size() > 0 && doc['Text.keyword'].value.indexOf(params.phrase)!=-1) count++;\n    return count;        \n    ",
   'params': {'phrase': 'represent'}}}}

In [60]:
# 统计特定词语的词频
data = {
    "_source": {
        "include": [
            "Name",
            "Year"
            ]
            },
    "from":0,
    "size":1,
    "sort": { "_id": { "order": "asc" }},
  "script_fields": x
}
headers = {'Content-Type': 'application/json'}

r = requests.get("http://localhost:9200/acl/_search", headers = headers, data = json.dumps(data))
json.loads(r.text)

{'took': 4,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3495, 'relation': 'eq'},
  'max_score': None,
  'hits': [{'_index': 'acl',
    '_type': '_doc',
    '_id': '000a817558bcecf98cdf4c991a5852d5',
    '_score': None,
    '_source': {'Year': '2014',
     'Name': 'A practical and linguistically-motivated approach to compositional distributional semantics'},
    'fields': {'key_words_frequency': [0]},
    'sort': ['000a817558bcecf98cdf4c991a5852d5']}]}}

In [ ]:
j = json.loads(r.text)
lines = []
jj = j['hits']['hits']
for item in jj:
    one_line = []
    address = item['_source']['address']
    balance = item['_source']['balance']
    temp = [item['fields'][key][0] for key in item['fields'].keys()]
    one_line.append(address)
    one_line.append(balance)
    one_line.extend(temp)
    lines.append(one_line)


In [ ]:
# 查找特定字段
data = {
  "query": { "match": { "Text": "approch" } }
}
headers = {'Content-Type': 'application/json'}

r = requests.get("http://localhost:9200/acl/_search", headers = headers, data = json.dumps(data))
json.loads(r.text)

### 失误处理

In [18]:
import json
with open('./pdf_es_json/NIPS_2017_2018_2019.json', 'r') as f1:
    rr = json.load(f1)
    print(len(rr))

3114


In [19]:
with open('./pdf_es_json/NIPS_2017_2018_2019.json', 'w') as f2:
    for i in rr:
        f2.write(i)

In [20]:
! head -n 1 ./pdf_es_json/NIPS_2017_2018_2019.json

{"index":{}}


### 手动处理错误

```
/home/d/papers_data/ACL/ACL_2018/Do\ Neural\ Network\ Cross-Modal\ Mappings\ Really\ Bridge\ Modalities\ .pdf
```

In [10]:
# 加md5处理; 将dumps文件转为ES bulk文件
from utils import curlmd5
import json

from utils import org_years_dict
json_file_folder = './pdf_es_json/old/'
json_file_paths = []
for org in org_years_dict.keys():
    if org == 'EMNLP': continue
    file_name = json_file_folder + f'{org}_' + '_'.join([year for year in org_years_dict[org]]) + '.json'
    json_file_paths.append(file_name)

for path in json_file_paths:
    
    with open(path, 'r') as f1:
        _content = f1.read().split('\n')
        print(len(_content))

    for i in range(0, len(_content)-1, 2):
        try:
            _name = json.loads(_content[i+1])["Name"]
            _new_start = '{"index":{"_id":\"' + curlmd5(_name) + '\"}}'
            _content[i] = _new_start
        except:
            print(i)
            print(_content[i+1])

    pdf_name = path.split('/')[-1]
    with open(f'./pdf_es_json/{pdf_name}', 'w') as f2:
        for i in _content:
            f2.write(i+'\n')

7007
9985
3967
2107
1633
6229


In [3]:
# 个别pdf无法读取; 设置超时时间
from pdfminer.layout import LAParams
from pdfminer.high_level import extract_text
laparams=LAParams(
    line_overlap=0.5,
    char_margin=2.0,
    line_margin=0.5,
    word_margin=0.1,
    boxes_flow=0.5,
    detect_vertical=True,
    all_texts=False,
)

In [5]:
!pip install func_timeout

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 51kB 3.0MB/s eta 0:00:011
  Created wheel for func-timeout: filename=func_timeout-4.3.5-cp37-none-any.whl size=15078 sha256=30e0fadaa26e089b4429b19f92411f5741a4fd435edbdb1a74a7c9fe7e5ca3bf
  Stored in directory: /root/.cache/pip/wheels/f6/17/26/63fc3d4477af2658d29baacf3279bafdf85222eae909744f32
Successfully built func-timeout


In [36]:
# 设置超时功能
from func_timeout import func_timeout, FunctionTimedOut

def _time_out_wrap(do_function, time_out:int , *kwargs):
    try:
        doitReturnValue = func_timeout(time_out, do_function, args = kwargs)
        return doitReturnValue
    except FunctionTimedOut:
        print ( f"{do_function.__name__}{kwargs} could not complete within {time_out} seconds and was terminated.")
    except:
        traceback.print_exc()

In [37]:
import time

def test(x1, x2):
    time.sleep(4)
    return f'oook {x1} {x2}'

In [38]:
_time_out_wrap(test, 3, '22', '222')

test ('22', '222') could not complete within 3 seconds and was terminated.


In [35]:
test.__name__

'test'

## test

In [79]:
path = './test.json'

with open(path, 'r') as f1:
    bulk_content_lines = f1.readlines()

url = f'http://localhost:9200/test/_bulk?pretty&refresh'
headers = {'Content-Type': 'application/json'}
for _index in range(0, len(bulk_content_lines), 2):
    data = ''.join(bulk_content_lines[_index: _index + 2]) + '\n'
    print(len(data))
    r = requests.post(url = url, headers = headers, data=data)
    j = json.loads(r.text)

322


In [80]:
j

{'took': 20,
 'errors': False,
 'items': [{'index': {'_index': 'test',
    '_type': '_doc',
    '_id': '7613d44ff8ad22f8a1a86612b4b40c61',
    '_version': 3,
    'result': 'updated',
    'forced_refresh': True,
    '_shards': {'total': 2, 'successful': 1, 'failed': 0},
    '_seq_no': 2,
    '_primary_term': 1,
    'status': 200}}]}

In [81]:
def _base_script(key_word:str):
    base_script = {}
    base_script["key_words_frequency"] = {}
    base_script["key_words_frequency"]["script"] = {}
    base_script["key_words_frequency"]["script"]["lang"] = "painless"
    base_script["key_words_frequency"]["script"]["source"] = f"""
    int count = 0;
    if(doc['Text.keyword'].size() > 0 && doc['Text.keyword'].value.indexOf(params.phrase)!=-1) count++;
    return count;        
    """
    base_script["key_words_frequency"]["script"]["params"] = {}
    base_script["key_words_frequency"]["script"]["params"]["phrase"] = key_word

    return base_script

In [83]:
# 统计特定词语的词频
x = _base_script("embeddings")
print(x)
data = {
    "_source": {
        "include": [
            "address",
            ]
            },
    "from":0,
    "size":10,
    "sort": { "_id": { "order": "asc" }},
  "script_fields": x
}
headers = {'Content-Type': 'application/json'}

r = requests.get("http://localhost:9200/test/_search", headers = headers, data = json.dumps(data))
json.loads(r.text)

{'key_words_frequency': {'script': {'lang': 'painless', 'source': "\n    int count = 0;\n    if(doc['Text.keyword'].size() > 0 && doc['Text.keyword'].value.indexOf(params.phrase)!=-1) count++;\n    return count;        \n    ", 'params': {'phrase': 'embeddings'}}}}


{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': None,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': '7613d44ff8ad22f8a1a86612b4b40c61',
    '_score': None,
    '_source': {},
    'fields': {'key_words_frequency': [1]},
    'sort': ['7613d44ff8ad22f8a1a86612b4b40c61']}]}}